In [90]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

In [91]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/_custom_/co_detr.py')

root='../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'updated_train.json' # train json 정보

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보

cfg.data.samples_per_gpu = 8

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_r50_fpn_1x_swin_trash'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [92]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [93]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4875, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3926  | 1 [Paper]     | 6230  | 2 [Paper pack]  | 890   | 3 [Metal]   | 849   | 4 [Glass]    | 977   |
| 5 [Plastic]       | 2873  | 6 [Styrofoam] | 1225  | 7 [Plastic bag] | 5143  | 8 [Battery] | 159   | 9 [Clothing] | 463   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [94]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-14 01:44:57,714 - mmdet - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-01-14 01:44:57,715 - mmcv - INFO - load model from: torchvision://resnet50
2024-01-14 01:44:57,716 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-01-14 01:44:57,806 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-01-14 01:44:57,821 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-14 01:44:57,846 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-01-14 01:44:57,852 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'overr

In [95]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-14 01:44:58,444 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-14 01:44:58,447 - mmdet - INFO - Start running, host: root@instance-5750, work_dir: /data/ephemeral/home/level2-objectdetection-cv-10/develop/yumin/mmdetection/work_dirs/cascade_rcnn_r50_fpn_1x_swin_trash
2024-01-14 01:44:58,447 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(A

KeyboardInterrupt: 